给定一个RPA流程需求描述，先让LLM重新描述执行步骤，然后根据执行步骤选择可能用到的函数名，再根据步骤描述和选定的函数生成最终的代码。

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA


openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
print(openai.api_type, openai.api_base, openai.api_version)


embeddings = OpenAIEmbeddings(model='text-embedding-ada-002',
                              deployment='emb0614',
                              openai_api_base=os.getenv("OPENAI_API_BASE"),
                              openai_api_type=os.getenv("OPENAI_API_TYPE"),
                              chunk_size=1)

docsearch = Chroma(persist_directory='apidocs', embedding_function=embeddings)

llm = ChatOpenAI(engine='test0406', model_name="gpt-3.5-turbo", temperature=0.3, max_tokens=4096)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


azure https://test0406.openai.azure.com/ 2023-03-15-preview


In [3]:
template_task = '''你是个非常优秀的RPA程序员，你的任务是根据RPA流程的需求描述和步骤描述，生成一个伪代码表示的RPA流程。
请一步步思考，确保正确理解实现该流程的所有必要条件，然后更新步骤描述，确保新的步骤描述包含了所有的上下文信息，以方便后面的代码。

流程描述: {description}

任务步骤：{steps}

重新描述实现该RPA流程的具体步骤，包括每个步骤的输入输出和实现功能：'''

prompt_task = PromptTemplate(input_variables=['description', 'steps'], template=template_task)
task_chain = LLMChain(llm=llm, prompt=prompt_task, output_key='revised')

In [4]:
template_draft = '''你是个非常优秀的RPA程序员，请根据RPA流程描述和实现步骤描述，生成一个用伪代码表示的RPA流程草稿。

流程描述: {description}

详细步骤：{revised}

生成的伪代码RPA流程是：'''

prompt_draft = PromptTemplate(input_variables=['description', 'revised'], template=template_draft)
draft_chain = LLMChain(llm=llm, prompt=prompt_draft, output_key='draft')

In [17]:
template_code = '''你是个非常优秀的RPA程序员，请根据RPA流程描述和流程草稿，实现一个由基本操作组成的伪代码RPA流程。
基本操作是指计算机能够执行的最简单任务，比如打开文件、保存文件、打开浏览器、点击按钮、修改文本等。

流程描述: {description}
流程草稿: {draft}

生成的RPA流程伪代码：'''

prompt_code = PromptTemplate(input_variables=['description', 'draft'], template=template_code)

code_chain = LLMChain(llm=llm, prompt=prompt_code, output_key='code')

In [18]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[task_chain, draft_chain, code_chain],
    input_variables=['description', 'steps'],
    # Here we return multiple variables
    output_variables=["revised", "draft", "code"],
    verbose=True)

In [19]:
task_description = '''通过RPA机器人，进入网银系统获取前一个工作日的资金流水数据，
按数据类型写入到模板表对应的银行列中，并下载前一工作日电子回单，生成文件清单并打印，
再发送资金日报及电子回单清单给对应的业务人员。
'''

task_steps = '''操作步骤及说明
1.	业务人员配置好银行账号数据表，和邮箱账号数据；
2.	读取账号配置表，根据账号信息自动登录网银系统；
3.	获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏；
4.	下载前一天电子回单、网银流水，按照固定规则命名（命名规则：公司_收/付_账号_日期，例：X公司_收_账号_20211222），并在日志记录表中写入记录；
5.	将日报excel及回、流水文件夹进行压缩；
6.	邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱；
7.	业务人员接收邮件文件，进行确认操作。
'''

In [20]:
result = overall_chain({'description': task_description, 'steps': task_steps})



> Entering new SequentialChain chain...

> Finished chain.


In [21]:
print(result['revised'])

1. 业务人员配置银行账号数据表和邮箱账号数据；
   输入：银行账号数据表、邮箱账号数据；
   输出：无；
   功能：配置银行账号和邮箱账号，为后续步骤做准备。

2. 读取账号配置表，根据账号信息自动登录网银系统；
   输入：银行账号数据表；
   输出：网银系统登录状态；
   功能：根据银行账号数据表中的账号信息，自动登录网银系统。

3. 获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏；
   输入：网银系统登录状态、资金日报表模板、银行账号数据表；
   输出：资金日报表模板对应银行账户余额栏；
   功能：登录网银系统，获取前一天历史余额数据，将其写入资金日报表模板对应银行账户余额栏。

4. 下载前一天电子回单、网银流水，按照固定规则命名（命名规则：公司_收/付_账号_日期，例：X公司_收_账号_20211222），并在日志记录表中写入记录；
   输入：网银系统登录状态、银行账号数据表、日志记录表；
   输出：前一天电子回单、网银流水文件夹；
   功能：登录网银系统，下载前一天电子回单和网银流水，按照固定规则命名，并在日志记录表中写入记录。

5. 将日报excel及回、流水文件夹进行压缩；
   输入：资金日报表模板、前一天电子回单、网银流水文件夹；
   输出：压缩文件夹；
   功能：将资金日报表模板、前一天电子回单、网银流水文件夹进行压缩。

6. 邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱；
   输入：压缩文件夹、邮箱账号数据、邮件内容；
   输出：邮件发送状态；
   功能：将运行的数据压缩文件发送到指定邮箱，同时发送运行异常提醒邮件。

7. 业务人员接收邮件文件，进行确认操作。
   输入：邮件内容；
   输出：确认操作状态；
   功能：业务人员接收邮件文件，进行确认操作。


In [22]:
print(result['draft'])

1. 配置银行账号数据表和邮箱账号数据
   Input: 银行账号数据表, 邮箱账号数据
   Output: 无
   Function: 配置银行账号和邮箱账号，为后续步骤做准备。

2. 自动登录网银系统
   Input: 银行账号数据表
   Output: 网银系统登录状态
   Function: 根据银行账号数据表中的账号信息，自动登录网银系统。

3. 获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏
   Input: 网银系统登录状态, 资金日报表模板, 银行账号数据表
   Output: 资金日报表模板对应银行账户余额栏
   Function: 登录网银系统，获取前一天历史余额数据，将其写入资金日报表模板对应银行账户余额栏。

4. 下载前一天电子回单、网银流水，按照固定规则命名，并在日志记录表中写入记录
   Input: 网银系统登录状态, 银行账号数据表, 日志记录表
   Output: 前一天电子回单、网银流水文件夹
   Function: 登录网银系统，下载前一天电子回单和网银流水，按照固定规则命名，并在日志记录表中写入记录。

5. 将日报excel及回、流水文件夹进行压缩
   Input: 资金日报表模板, 前一天电子回单、网银流水文件夹
   Output: 压缩文件夹
   Function: 将资金日报表模板、前一天电子回单、网银流水文件夹进行压缩。

6. 邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱
   Input: 压缩文件夹, 邮箱账号数据, 邮件内容
   Output: 邮件发送状态
   Function: 将运行的数据压缩文件发送到指定邮箱，同时发送运行异常提醒邮件。

7. 业务人员接收邮件文件，进行确认操作
   Input: 邮件内容
   Output: 确认操作状态
   Function: 业务人员接收邮件文件，进行确认操作。


In [23]:
print(result['code'])

1. 配置银行账号数据表和邮箱账号数据
   bank_account_data = read_excel('bank_account_data.xlsx')
   email_account_data = read_excel('email_account_data.xlsx')

2. 自动登录网银系统
   login_to_bank(bank_account_data)

3. 获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏
   daily_report_template = read_excel('daily_report_template.xlsx')
   balance_data = get_balance_data_from_bank()
   write_balance_data_to_report_template(daily_report_template, balance_data)

4. 下载前一天电子回单、网银流水，按照固定规则命名，并在日志记录表中写入记录
   download_folder = create_download_folder()
   download_electronic_receipt(download_folder)
   download_bank_statement(download_folder)
   write_download_log(download_folder)

5. 将日报excel及回、流水文件夹进行压缩
   compressed_folder = compress_files(daily_report_template, download_folder)

6. 邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱
   email_content = create_email_content(compressed_folder)
   send_email(email_account_data, email_content)

7. 业务人员接收邮件文件，进行确认操作
   confirm_email_received()


In [26]:
# with open('patents/requirements-test4.txt', 'w') as f:
#     f.write('requirements: ' + result['requirements'])
#     f.write('\n\nfeature: ' + result['feature'])
#     f.write('\n\nsolution: ' + result['solution'])
#     f.write('\n\nselection: ' + result['selection'])

In [25]:
steps = result['code'].split('\n\n')
steps[:3]

["1. 配置银行账号数据表和邮箱账号数据\n   bank_account_data = read_excel('bank_account_data.xlsx')\n   email_account_data = read_excel('email_account_data.xlsx')",
 '2. 自动登录网银系统\n   login_to_bank(bank_account_data)',
 "3. 获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏\n   daily_report_template = read_excel('daily_report_template.xlsx')\n   balance_data = get_balance_data_from_bank()\n   write_balance_data_to_report_template(daily_report_template, balance_data)"]

In [29]:
template_generate = """你是一个专业的RPA开发人员，请根据RPA库函数描述，为当前步骤选择合适的RPA库函数。
如果不存在实现这个步骤相关的函数，或者步骤描述不够清晰无法判断，则回答"no"；除此之外，不需要其他的解释或者说明。

相关函数: {context}

当前步骤：{question}

相关的函数列表:"""

prompt_generate = PromptTemplate(
    template=template_generate, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": prompt_generate}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [30]:
functions = []
for i in range(len(steps)):
    result = qa.run(steps[i])
    functions.append(result)
    print(f'>> {steps[i]} \n {result} \n')

>> 1. 配置银行账号数据表和邮箱账号数据
   bank_account_data = read_excel('bank_account_data.xlsx')
   email_account_data = read_excel('email_account_data.xlsx') 
 read_excel 

>> 2. 自动登录网银系统
   login_to_bank(bank_account_data) 
 no 

>> 3. 获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏
   daily_report_template = read_excel('daily_report_template.xlsx')
   balance_data = get_balance_data_from_bank()
   write_balance_data_to_report_template(daily_report_template, balance_data) 
 read_excel, write_row_to_excel, no 

>> 4. 下载前一天电子回单、网银流水，按照固定规则命名，并在日志记录表中写入记录
   download_folder = create_download_folder()
   download_electronic_receipt(download_folder)
   download_bank_statement(download_folder)
   write_download_log(download_folder) 
 batch_download_files_from_server, no, no, no 

>> 5. 将日报excel及回、流水文件夹进行压缩
   compressed_folder = compress_files(daily_report_template, download_folder) 
 compressPDF 

>> 6. 邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱
   email_content = create_email_content(compressed_folder)
   send_email(email_account_d